#데이터셋, 데이터로더에서 난항 겪음 -> 일단은 keras에서 데이터 불러와서 numpy 거쳐서 데이터셋으로 만듦

In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.autograd import Variable
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

# 데이터 확인

In [2]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)

print('훈련용 리뷰 개수 : {}'.format(len(X_train)))
print('테스트용 리뷰 개수 : {}'.format(len(X_test)))
num_classes = len(set(y_train))
print('카테고리 : {}'.format(num_classes))


17464789/17464789 [==============================] - 2s 0us/step
훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000
카테고리 : 2


In [3]:
print(type(X_train), type(y_train), type(X_test), type(y_test))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [4]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(25000,) (25000,) (25000,) (25000,)


In [5]:
print(f"X_train: {X_train}\n")
print(f"y_train: {y_train}")

X_train: [list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8255, 78, 228, 5, 

#데이터 전처리

패딩

In [6]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

텐서로 변환

In [7]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

데이터셋으로 변환

In [8]:
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.x_data.shape[0]

In [9]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

데이터 형태 확인

In [10]:
train_dataset[0][0].shape

torch.Size([500])

In [11]:
(train_dataset[0][1])

tensor(1)

데이터로더로 변환

In [12]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

In [13]:
for (X, y) in train_dataloader:
  print(X.shape)
  print(len(y))
  break

torch.Size([64, 500])
64


#모델 만들기

GPU 사용

In [20]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


임베딩레이어, GRU, 손실함수, 옵티마이저 정의

In [21]:
embedding_dim = 100
embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = 0)

In [22]:
class GRUModel(nn.Module) :
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True) :
        super(GRUModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(input_size=input_dim, hidden_size=hidden_dim, num_layers=layer_dim, bidirectional=False, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sig = nn.Sigmoid()

    def forward(self, x) :
      embedded = self.embedding(x)
      out, hidden = self.gru(embedded)
      last_hidden = hidden.squeeze(0)
      out = self.fc(last_hidden)
      out = torch.flatten(out)
      out = self.sig(out)
      return out

In [56]:
input_dim = 100
hidden_dim = 128
layer_dim = 1
output_dim = 1
learning_rate = 1e-3

model = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)
loss_fn = nn.BCELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

In [57]:
model = model.to(device)
loss_fn = loss_fn.to(device)

차원 맞추기가 힘들다..... GRU에서 마지막 step 출력값 모아서 pred에 넣고 싶음 / 해결

속도 너무 느리다 / 해결

accuracy 평가 어떻게 하지? / 해결

#학습

In [58]:
def train_loop(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred.to(torch.float), y.to(torch.float))

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred.to(torch.float), y.to(torch.float)).item()
            pred = pred - 0.5
            value = torch.Tensor([0])
            value = value.to(device)
            torch.heaviside(pred, value)
            correct += (pred == y).sum()

    test_loss /= num_batches
    #correct /= size
    print(f"Test Error: \n Accuracy: {correct}%, Avg loss: {test_loss:>8f} \n")

In [62]:
epochs = 7
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.003647  [   64/25000]
loss: 0.072072  [ 6464/25000]
loss: 0.005876  [12864/25000]
loss: 0.034583  [19264/25000]
Test Error: 
 Accuracy: 0%, Avg loss: 0.627782 

Epoch 2
-------------------------------
loss: 0.001773  [   64/25000]
loss: 0.001854  [ 6464/25000]
loss: 0.014828  [12864/25000]
loss: 0.002329  [19264/25000]
Test Error: 
 Accuracy: 0%, Avg loss: 0.650301 

Epoch 3
-------------------------------
loss: 0.002286  [   64/25000]
loss: 0.000617  [ 6464/25000]
loss: 0.003852  [12864/25000]
loss: 0.000155  [19264/25000]
Test Error: 
 Accuracy: 0%, Avg loss: 0.610878 

Epoch 4
-------------------------------
loss: 0.001282  [   64/25000]
loss: 0.037804  [ 6464/25000]
loss: 0.001982  [12864/25000]
loss: 0.000601  [19264/25000]
Test Error: 
 Accuracy: 0%, Avg loss: 0.654265 

Epoch 5
-------------------------------
loss: 0.005236  [   64/25000]
loss: 0.002296  [ 6464/25000]
loss: 0.002478  [12864/25000]
loss: 0.000706  [19264/25000]
Test

In [63]:
value = torch.Tensor([0.5])
value = value.to(device)
i = 0
accuracy = 0
for X , y in test_dataloader:
  X = X.to(device)
  y = y.to(device)
  pred = model(X)
  pred = pred - 0.5
  pred = torch.heaviside(pred, value)
  i += 1
  accuracy += (pred == y).sum() / len(X)
accuracy /= i
print(f"accuracy: {accuracy * 100:>2f}%")

accuracy: 88.029091%


#저장/로드

설정

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


저장

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/second_GRU.pth')

불러오기

In [ ]:
model = torch.load('/content/gdrive/MyDrive/first_GRU.pth')

In [ ]:
value = torch.Tensor([0.5])
i = 0
accuracy = 0
for X , y in test_dataloader:
  pred = model(X)
  pred = pred - 0.5
  pred = torch.heaviside(pred, value)
  i += 1
  accuracy += (pred == y).sum() / len(X)
accuracy /= i
print(f"accuracy: {accuracy * 100:>2f}%")

TypeError: 'collections.OrderedDict' object is not callable